In [1]:
import torch
from datasets import load_dataset
from datasets import load_from_disk
import random


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
#定义数据集
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, split):
        # dataset = load_dataset(path='lansinuote/ChnSentiCorp', split=split)
        dataset = load_from_disk('./data/ChnSentiCorp')['%s' % split]
        
        def f(data):
            return len(data['text']) > 40

        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset['text'][i]
        
        # print('原始数据: ', text)
        # label = self.dataset['label'][i]
        #切分一句话为前半句和后半句
        sentence1 = text[:20]
        sentence2 = text[20:]
        label = 0

        # 有一半的概率把后半句替换为一句无关的话
        if random.randint(0, 1) == 0:
            j = random.randint(0, len(self.dataset) - 1)
            # sentence2 = self.dataset[j]['text'][20:40]
            sentence2 = self.dataset['text'][j][20:]
            label = 1

        return sentence1, sentence2, label

# 加载
dataset = Dataset('train')

Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/train/cache-248dcc552a1bfcb6.arrow


In [4]:
for i in range(5):
    print(dataset[i])
    print("")

('选择珠江花园的原因就是方便，有电动扶梯直', '接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 0)

('15.4寸笔记本的键盘确实爽，基本跟台式', '停车场（如果住在古镇里面，白天是不允许把车开进去的）。 这个宾馆的服务员都说自己的餐厅做的当地小吃好吃，实在不敢苟同，份量少，味道也不地道，价格却不低，建议重视当地美食的朋友不要在宾馆的餐厅就餐，会对山西的小吃产生错误印象的。', 1)

('1.接电源没有几分钟,电源适配器热的不行', '述所见所闻，亲身感想，以及针砭时事。让人感到耳目一新。里面主要有法国和意大利两部分，个人觉得意大利的那部分写的要比法国的要好得很多，感觉比较像个住客（原来他连房子都买了）以主观深入的心情叙述见闻，而且可以感觉到他是真正喜欢意大利这个地方。而法国的部分感觉像是交作业的游记，完全像是一个短期的游客、一个过客。', 1)

('今天才知道这书还有第6卷,真有点郁闷:为', '什么同一套书有两种版本呢?当当网是不是该跟出版社商量商量,单独出个第6卷,让我们的孩子不会有所遗憾。', 0)

('机器背面似乎被撕了张什么标签，残胶还在。', '挺好。在当当网上按作者姓名查询找到的一直缺货，后来发现这本没有“模样”的同名书，是同一出版社出版的，就抱着试试看的想法订购了一本，结果正式那一直表示“暂时缺货”的、我想订购的书。希望当当尽快完善商品信息，以方便读者订阅。书很好，相对浅显易懂地介绍了黄帝内经的核心内容。但对于初次接触这些知识的人而言，似乎需要多读几遍才能加深理解。', 1)



In [5]:
# 测试拼接的数据

sentence1, sentence2, label = dataset[3]

len(dataset), sentence1, sentence2, label

(8001,
 '今天才知道这书还有第6卷,真有点郁闷:为',
 '什么同一套书有两种版本呢?当当网是不是该跟出版社商量商量,单独出个第6卷,让我们的孩子不会有所遗憾。',
 0)

In [6]:
from transformers import BertTokenizer
from transformers import BertModel


# 加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

print(token)


# 加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese').to(device)

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
def collate_fn(data):
    sents = [i[:2] for i in data]
    labels = [i[2] for i in data]
    
    # print("sents: ", sents)

    # 编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=300,
                                   return_tensors='pt',
                                   return_length=True,
                                   add_special_tokens=True)

    # input_ids: 编码之后的数字
    # attention_mask: 是补零的位置是0,其他位置是1
    # token_type_ids: 第一个句子和特殊符号的位置是0,第二个句子的位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels


## 数据加载器

In [9]:

loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=8,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)


for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    print(len(loader))
    print(token.decode(input_ids[0]))
    break

input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

1000
[CLS] 3 星 的 硬 件, 4 星 的 外 观, 5 星 的 服 务 房 间 [SEP] 设 施 比 较 陈 旧, 但 服 务 非 常 的 好, 头 一 次 碰 到 3 星 的 酒 店 里 任 何 一 个 服 务 人 员 见 到 你 都 会 微 笑 问 好 的, 如 果 设 施 再 好 些 是 个 不 错 的 酒 店 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

(torch.Size([8, 300]),
 torch.Size([8, 300]),
 torch.Size([8, 300]),
 tensor([0, 1, 0, 0, 1, 1, 0, 0]))

In [10]:
# 不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

In [11]:
# 定义下游任务模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out

In [12]:
model = Model()

# 模型转移到GPU上
model.to(device)

Model(
  (fc): Linear(in_features=768, out_features=2, bias=True)
)

In [13]:
from transformers import AdamW


# 训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()


model.train()


for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)
    labels = labels.to(device)

    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
    
    loss = criterion(out, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i % 5 == 0:
        labels = labels.cpu()
        out = out.argmax(dim=1).cpu()
        
        accuracy = (out == labels).sum().item() / len(labels)
        print(i, loss.item(), accuracy)

    if i == 300:
        break

/home/mylady/.virtualenvs/dl-pytorch/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 0.6995781660079956 0.5


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


5 0.5784845352172852 0.75


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

285 0.3200768232345581 1.0


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


290 0.4685365855693817 0.875
295 0.46032387018203735 0.875


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


300 0.38100236654281616 0.875


In [14]:
#测试
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)
    
    print('测试数据长度: ', len(loader_test))
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        
        #if i == 5:
        #    break
            
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
            pass
        pred = out.argmax(dim=1)
        
        # 转移到cpu上
        pred = pred.cpu()
        labels = labels.cpu()
        
        correct += (pred == labels).sum().item()
        total += len(labels)
        
        if i // 200 == 0:
            print(i, 'acc: ', correct / total)
            pass

    print('acc: ', correct / total)


In [15]:


test()

Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/test/cache-c06d6d85e943fbe5.arrow


测试数据长度:  30
0 acc:  0.90625


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


1 acc:  0.9375


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

23 acc:  0.8919270833333334


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


24 acc:  0.88875
25 acc:  0.8870192307692307


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


26 acc:  0.8842592592592593
27 acc:  0.8828125
28 acc:  0.8857758620689655
29 acc:  0.8833333333333333
acc:  0.8833333333333333


In [17]:
# 保存模型

# 保存

model_save_path = 'chinese_infer_mission_2023_4_10.pt'
torch.save(model.state_dict(),  model_save_path)  # 推荐的文件后缀名是pt或pth
